## Download Quads (satellite images) from Mosiacs (a map of a given time period) Using Planet Basemaps API
---

The base code was partially taken / inspired from the Planet API documentation and examples: https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/quad_tutorial/download_quads.ipynb & https://developers.planet.com/docs/basemaps/

In [1]:
import os
import requests
import json
from urllib.request import urlopen
from shutil import copyfileobj

In [3]:
# if your Planet API Key is not set as an environment variable, you can paste it below by modifying this to: API_KEY = os.environ.get('PL_API_KEY', 'your_api_key')
API_KEY = os.environ.get('PL_API_KEY')

# Setup Planet Data API base URL
URL = "https://api.planet.com/basemaps/v1/mosaics"

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

In [4]:
#Test if API is working on a simple example

#set params for search using name of mosaic
parameters = {
    "name__is" :"planet_medres_normalized_analytic_2023-10_mosaic" 
}
#make get request to access mosaic from basemaps API
res = session.get(URL, params = parameters)
mosaic = res.json()

#get id
mosaic_id = mosaic['mosaics'][0]['id']
#get bbox for entire mosaic
mosaic_bbox = mosaic['mosaics'][0]['bbox']
#converting bbox to string for search params
string_bbox = ','.join(map(str, mosaic_bbox))

#search for mosaic quad using AOI
search_parameters = {
    'bbox': string_bbox,
    'minimal': True
}
quad_id = '628-961'
#accessing quads using metadata from mosaic
quads_url = "{}/{}/quads/{}".format(URL, mosaic_id, quad_id)
#quads_url = "{}/{}/quads/".format(URL, mosaic_id)
res = session.get(quads_url, params=search_parameters, stream=True)

quads = res.json()

A set of helper functions are defined below to automate querying the API:
- `get_quads_per_mosaic` takes a list of quads and a mosaic, queries the API and returns the results in a dictionary
- `mosaic_quad_dictionary` iterates over a list of mosaics and implements `get_quads_per_mosaic` for each, returns a dictionary of mosaics-quad API call results
- `download_thumbnails` uses the API response dictionary generated and downloads thumbnails
- `download_images` downloads the full quality images

In [5]:
def get_quads_per_mosaic(quads,mosaics):
    '''
    Takes a list and a string as input parameters: a list of quads, defined as, for example ['677-938', '679-938'], 
    and a string of a mosaics, like 'planet_medres_visual_2023-06_mosaic'
    Returns a dictionary of the results for the API call for each quad in the mosaic
    '''
    #set params for search using name of mosaic
    parameters = {
        "name__is" :mosaics 
    }
    #make get request to access mosaic from basemaps API
    res = session.get(URL, params = parameters)
    mosaic = res.json()

    #get id
    mosaic_id = mosaic['mosaics'][0]['id']
    #get bbox for entire mosaic
    mosaic_bbox = mosaic['mosaics'][0]['bbox']
    #converting bbox to string for search params
    string_bbox = ','.join(map(str, mosaic_bbox))

    #search for mosaic quad using AOI
    search_parameters = {
        'bbox': string_bbox,
        'minimal': True
    }

    # Dictionary to store quad data
    quads_data_dict = {}

    # Iterate over the quads
    for quad_id in quads:
        quads_url = "{}/{}/quads/{}".format(URL, mosaic_id, quad_id)
        res = session.get(quads_url, params=search_parameters, stream=True)
        quad_data = res.json()
        quads_data_dict[quad_id] = quad_data

    return quads_data_dict






In [6]:
def mosaic_quad_dictionary(quad_list, mosaic_list):
    master_dict = {}
    for m in mosaic_list:
        quads_data = get_quads_per_mosaic(quad_list, m)
        master_dict[m] = quads_data
    return master_dict

In [7]:
def download_thumbnails(master_data, save_dir, prints=False):
    """
    Downloads thumbnails for each quad in each mosaic and saves them to the specified directory.

    :param master_data: A multi-level dictionary with mosaic names and quads as keys and the results of the API query as values (includes link to thumbnail or download).
    :param save_dir: The directory where thumbnails will be saved. 
    """

    # Iterate over each mosaic and its quads
    for mosaic_name, quads in master_data.items():
        for quad_id, quad_data in quads.items():
            # Check if the '_links' and 'thumbnail' keys exist
            if '_links' in quad_data and 'thumbnail' in quad_data['_links']:
                link = quad_data['_links']['thumbnail']
                # Construct a filename using both mosaic name and quad id
                filename = f"{mosaic_name}_{quad_id}.png"
                filepath = os.path.join(save_dir, filename)

                try:
                    with urlopen(link) as in_stream, open(filepath, 'wb') as out_file:
                        copyfileobj(in_stream, out_file)
                        if prints==True:
                            print(f"Downloaded thumbnail for {filename}")
                except Exception as e:
                    print(f"Error downloading {filename}: {e}")

In [8]:
def download_images(master_data, save_dir, prints=False):
    """
    Downloads thumbnails for each quad in each mosaic and saves them to the specified directory.

    :param master_data: A multi-level dictionary with mosaic names and quads as keys and the results of the API query as values (includes link to thumbnail or download).
    :param save_dir: The directory where thumbnails will be saved.
    """

    # Iterate over each mosaic and its quads
    for mosaic_name, quads in master_data.items():
        for quad_id, quad_data in quads.items():
            # Check if the '_links' and 'thumbnail' keys exist
            if '_links' in quad_data and 'download' in quad_data['_links']:
                link = quad_data['_links']['download']
                # Construct a filename using both mosaic name and quad id
                filename = f"{mosaic_name}_{quad_id}.tif"
                filepath = os.path.join(save_dir, filename)

                try:
                    with urlopen(link) as in_stream, open(filepath, 'wb') as out_file:
                        copyfileobj(in_stream, out_file)
                        if prints == True:
                            print(f"Downloaded thumbnail for {filename}")
                except Exception as e:
                    print(f"Error downloading {filename}: {e}")

## Data collection for the analysis

The 36 quad ID-s with download for 2 periods and thumbnails for 1 period per year

In [9]:
quad_list = ['672-933', 
'673-933', 
'674-933', 
'675-933', 
'676-933', 
'677-933', 
'672-932', 
'673-932', 
'674-932', 
'675-932', 
'676-932', 
'677-932', 
'672-931', 
'673-931', 
'674-931', 
'675-931', 
'676-931', 
'677-931', 
'672-930', 
'673-930', 
'674-930', 
'675-930', 
'676-930', 
'677-930', 
'672-929', 
'673-929', 
'674-929', 
'675-929', 
'676-929', 
'677-929', 
'672-928', 
'673-928', 
'674-928', 
'675-928', 
'676-928', 
'677-928']

mosaic_pres_2yr_anl = ['planet_medres_normalized_analytic_2023-08_mosaic','planet_medres_normalized_analytic_2017-06_2017-11_mosaic']

mosaic_list_anl = [
                         'planet_medres_normalized_analytic_2023-08_mosaic',
                         'planet_medres_normalized_analytic_2022-07_mosaic',
                         'planet_medres_normalized_analytic_2021-07_mosaic',
                         'planet_medres_normalized_analytic_2020-06_2020-08_mosaic',
                         'planet_medres_normalized_analytic_2019-06_2019-11_mosaic',
                         'planet_medres_normalized_analytic_2018-06_2018-11_mosaic',
                         'planet_medres_normalized_analytic_2017-06_2017-11_mosaic',
                         'planet_medres_normalized_analytic_2016-06_2016-11_mosaic'

]

In [10]:
#functions to switch between visual and normalized data
def normalized_to_viz(mosaic_list):
    list = []
    for m in mosaic_list:
        mnew = m.replace('_normalized_analytic_','_visual_')
        list.append(mnew)
    return list

#create the visual lists
def viz_to_normalized(mosaic_list):
    list = []
    for m in mosaic_list:
        mnew = m.replace('_visual_','_normalized_analytic_')
        list.append(mnew)
    return list

In [11]:

pres_yearly_anl = mosaic_quad_dictionary(quad_list, mosaic_pres_2yr_anl)
pres_allyears_anl = mosaic_quad_dictionary(quad_list, mosaic_list_anl)

In [36]:
#check the lenght - If downloading, lenght x 40-100 MB is your download size
for m in pres_yearly_anl:
    print(len(pres_yearly_anl[m]))

36
36


Given the size of the full-slace images, we will work with png images for all periods, and only work with .tiff images for the start and end time (2017 and 2023) as a separate analysis

In [54]:
#download_thumbnails(pres_allyears_anl, 'presentation_data/2yr_anl')
#download_images(pres_yearly_anl, 'presentation_data/2years_anl')